Testklient for nye vegnettfunksjoner for stedfesting m.m. 

In [2]:
import requests
import json
import hvplot 
import pandas as pd
import datetime
import xmltodict

# Geodata-stuff 
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point
import hvplot.pandas
from holoviews import opts
import holoviews as hv
import cartopy.crs as ccrs

# støtte for scroll med mus
opts.defaults( 
    opts.Overlay(
        width=400, height=600,
        active_tools=['wheel_zoom']
))

In [3]:
# Omsette kall til visveginfo => geopandas
def hentvisveginfo( startEasting , startNorthing, endEasting, endNorthing, topologylevel='Overview'):
    url = 'https://visveginfo.opentns.org/RoadInfoService/GetRoadDataAlongRouteBetweenLocations'
    params = { 'viewDate' : str( datetime.date.today() ), 
               'TopologyLevel' : topologylevel, 
                 'startEasting' : startEasting, 
                 'startNorthing': startNorthing, 
                 'endEasting' : endEasting, 
                 'endNorthing' : endNorthing }
    r = requests.get( url, params)
    
    data = xmltodict.parse( r.text )
    
    if not 'RoadLineReference' in data['RouteWithData']['Route'].keys(): 
        return None 
    
    
    mylist = []
    if isinstance( data['RouteWithData']['Route']['RoadLineReference'], list ): 
        for link in data['RouteWithData']['Route']['RoadLineReference']:
            mylist.append( vvisegment2dict( link ))
    else: 
        mylist.append( vvisegment2dict( data['RouteWithData']['Route']['RoadLineReference']  ) )
    
    mydf = pd.DataFrame( mylist)
    mygdf = gpd.GeoDataFrame( mydf, geometry='geometry')
    mygdf.crs = { 'init' : 'epsg:25833'}
    return mygdf
    
def vvisegment2dict( link):
    
    start = round( float( link['FromMeasure'] ), 8 )
    slutt = round( float( link['ToMeasure']   ), 8 )
    mydict = { 'vegref' : str( link['County']).zfill(2) + str( link['Municipality'] ).zfill(2) + ' ' + \
                          link['RoadCategory'].upper() + link['RoadStatus'].lower() + str( link['RoadNumber']) + \
                          'hp' + str( link['RoadNumberSegment']) + ' m' + \
                          str(link['RoadNumberSegmentStartDistance'] ) + '-' + \
                          str( link['RoadNumberSegmentEndDistance']), 
               'veglenkesekvensid' : link['ReflinkOID'], 
               'startposisjon' : start, 
               'sluttposisjon' : slutt, 
               'kortform'      : str( start ) + '-' + str( slutt ) + \
                                  '@' + str( link['ReflinkOID']), 
               'geometry'      :  wkt.loads( link['WKTGeometry']  )
            }
    mydict['lengde'] = mydict['geometry'].length
    
    return mydict   

In [4]:
# Går fra punkt-koordinat til geodataframe
def koord2gdf( startEasting , startNorthing, endEasting, endNorthing, navn='Rute'):
    """
    Omsetter start- og sluttpunkt til geodataframe
    """
    mylist = []
    mylist.append( punkt2dict( startEasting, startNorthing, ' '.join( [ navn, 'start' ] )  ) )
    mylist.append( punkt2dict( endEasting, endNorthing, ' '.join([ navn, 'slutt' ])  ) )    
    
    tempdf = pd.DataFrame( mylist )
    mygdf = gpd.GeoDataFrame( tempdf, geometry='geometry')
    return mygdf 
    
def punkt2dict(x, y, navn  ):
    """
    Omsetter punkt + navn til dictionary med shapely POINT 
    """
    retval =  { 'navn' : navn, 'geometry' : Point( x, y)  }
    return retval 

In [5]:
# Henter rute fra nytt rute-endepunkt 
def hentstrekning( startEasting , startNorthing, endEasting, endNorthing, **kwargs):
    url = 'https://nvdbw01.kantega.no/nvdb/api/v3/beta/vegnett/rute'
    # url = 'https://apilesv3.utv.atlas.vegvesen.no/beta/vegnett/rute'
    params = { 'start' :  str(startEasting) + ',' + str(startNorthing ), 
                'slutt' : str(endEasting)   + ',' + str(endNorthing), 
                 **kwargs }
    
    r = requests.get( url, params=params )
    print( r.url )
    
    data = r.json()
    linklist = []
    
    for link in data: 
        linklist.append( v3segment2dict( link ) )
        
    mydf = pd.DataFrame( linklist)
    mylinkgdf = gpd.GeoDataFrame( mydf, geometry='geometry')
    
    return mylinkgdf

def v3punkt2dict( punkt, navn='navn' ): 
    punkt['geometry'] = wkt.loads( punkt['geometri']['wkt'] )
    punkt['vegref'] = punkt['vegsystemreferanse']['kortform']
    punkt['navn'] = navn
    punkt['kortform'] = punkt['veglenkesekvens']['kortform']
    
    return punkt

def v3segment2dict( link): 
    link['geometry'] = wkt.loads( link['geometri']['wkt'] )
    link['vegref'] = link['vegsystemreferanse']['kortform']
    return link

In [6]:
# Visveginfo eksempel 
x1 = -2162.328125
y1 = 6652822 
x2 = 3915.72
y2 = 6653972.18


# Breidablikkveien, hver side av rundkjøring
x1 = 269303.11
y1 = 7039794.91
x2 = 269218.65
y2 = 7039853.75


# Bratsbergveien, kort bit
x1 = 270808.18
y1 = 7037350.54
x2 = 270820.74
y2 = 7037339.47

# Kantega eksempel 
x1 = 225205.2985792723
y1 = 6564325.697602992
x2 = 225256.7469531842
y2 = 6564356.513124289

# Breidablikkveien, hver side av rundkjøring
x1 = 269303.11
y1 = 7039794.91
x2 = 269218.65
y2 = 7039853.75

# Sandefjord rundkj
x1=227302.03432728566
y1=6564019.459212849
x2=227381.15034680913
y2=6563997.188293161



# Henter ruteforslag 

Henter ruteforslag fra visveginfo og den nye tjenesten

In [7]:
vvirute = hentvisveginfo( x1, y1, x2, y2)
nyrute_vt = hentstrekning( x1, y1, x2, y2, omkrets=100, detaljerte_lenker=False)
nyrute_kb = hentstrekning( x1, y1, x2, y2, omkrets=100, detaljerte_lenker=True)
orginalpunkt = koord2gdf( x1, y1, x2, y2)


https://nvdbw01.kantega.no/nvdb/api/v3/beta/vegnett/rute?start=227302.03432728566%2C6564019.459212849&slutt=227381.15034680913%2C6563997.188293161&omkrets=100&detaljerte_lenker=False
https://nvdbw01.kantega.no/nvdb/api/v3/beta/vegnett/rute?start=227302.03432728566%2C6564019.459212849&slutt=227381.15034680913%2C6563997.188293161&omkrets=100&detaljerte_lenker=True


In [8]:
vvirute.hvplot(  crs=ccrs.UTM(zone=33), tiles='OSM', line_width=10, c='kortform', 
            cmap='Category10', legend=False, title='Visveginfo' ) * \
orginalpunkt.hvplot( crs=ccrs.UTM(zone=33), size=100, color='g') + \
nyrute_vt.hvplot( crs=ccrs.UTM(zone=33), tiles='OSM', line_width=10, c='kortform', \
                 cmap='Category10', legend=False, title='NVDB vegtrasé' ) * \
orginalpunkt.hvplot( crs=ccrs.UTM(zone=33), size=100, color='g') + \
nyrute_kb.hvplot( crs=ccrs.UTM(zone=33), tiles='OSM', line_width=10, c='kortform', \
                 cmap='Category10', legend=False, title='NVDB detaljerte lenker' ) * \
orginalpunkt.hvplot( crs=ccrs.UTM(zone=33), size=100, color='g')

# nyepunkt.hvplot( crs=ccrs.UTM(zone=33), size=300, color='r')

:Layout
   .Overlay.I   :Overlay
      .Tiles.I  :Tiles   [x,y]
      .Path.I   :Path   [Longitude,Latitude]   (kortform)
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.II  :Overlay
      .Tiles.I  :Tiles   [x,y]
      .Path.I   :Path   [Longitude,Latitude]   (kortform)
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.III :Overlay
      .Tiles.I  :Tiles   [x,y]
      .Path.I   :Path   [Longitude,Latitude]   (kortform)
      .Points.I :Points   [Longitude,Latitude]

# Få ut hele rundkjøringen? 

For stedfesting av abstrakte strekningsobjekt ønsker vi som regel å stedfeste på **hele** rundkjøringen. Mulighet til å få ut hele rundkjøringen? 

In [12]:
nyrute_vt[['typeVeg', 'vegref']]

,typeVeg,vegref
0,Kanalisert veg,FV260 S8D1 m547-575
1,Rundkjøring,FV260 S8D1 m547 KD1 m28-34
2,Rundkjøring,FV260 S8D1 m547 KD1 m22-28
3,Rundkjøring,FV260 S8D1 m547 KD1 m16-22
4,Rundkjøring,FV260 S8D1 m547 KD1 m9-16
5,Rundkjøring,FV260 S8D1 m547 KD1 m0-9
6,Rundkjøring,FV260 S8D1 m547 KD1 m74-78
7,Kanalisert veg,FV260 S8D1 m519-547
